# Convolutional Neural Network (CNN) Image Recognition Assignment

This notebook contains the full assignment: dataset selection, model implementation, training, evaluation, plots, and analysis.

**Dataset chosen:** CIFAR-10 (available from Keras).

**What the CNN will recognize:** 10 classes of tiny (32x32) images: airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck.

## Problem statement

Build and train a CNN to classify images from the CIFAR-10 dataset into their correct class. Evaluate model accuracy and loss on training and test sets, plot training curves, and provide analysis of the findings.

## Algorithm / Approach

1. Load CIFAR-10 dataset.
2. Preprocess: normalize images to [0,1], one-hot encode labels.
3. Build CNN: multiple Conv2D + ReLU + MaxPooling layers, Flatten, Dense (ReLU), Output Dense (Softmax).
4. Compile with categorical_crossentropy, adam optimizer, and accuracy metric.
5. Train for >= 50 epochs with validation split and callbacks.
6. Evaluate on test set, plot loss/accuracy, show confusion matrix and sample predictions.

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

print('TensorFlow version:', tf.__version__)

In [ ]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)

In [ ]:
# Preprocess: normalize and one-hot encode
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

print('Prepared x_train shape:', x_train.shape, 'dtype:', x_train.dtype)
print('Prepared y_train shape:', y_train_cat.shape)

In [ ]:
# Display a few sample images
plt.figure(figsize=(8,4))
for i in range(10):
    plt.subplot(2,5,i+1)
    plt.imshow(x_train[i])
    plt.title(class_names[int(y_train[i])])
    plt.axis('off')
plt.tight_layout()
plt.show()

## Model construction

We'll build a sequential CNN with three Conv2D+MaxPooling blocks, then flatten and add dense layers. We'll use ReLU activations and Softmax for the output. Max pooling is used to downsample and keep the strongest features (translation invariance).

In [ ]:
def build_cnn(input_shape=(32,32,3), num_classes=10):
    model = models.Sequential()
    # First conv block
    model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2,2)))
    # Second conv block
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.MaxPooling2D((2,2)))
    # Third conv block
    model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.MaxPooling2D((2,2)))
    # Flatten and dense layers
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

model = build_cnn()
model.summary()

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
es = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
mc = ModelCheckpoint('best_cnn_cifar10.h5', monitor='val_accuracy', save_best_only=True)

# Train
history = model.fit(x_train, y_train_cat, epochs=50, batch_size=64, validation_split=0.15, callbacks=[es, mc])

In [ ]:
# Evaluate on test set
test_loss, test_acc = model.evaluate(x_test, y_test_cat, verbose=2)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

In [ ]:
# Plot training history
fig, ax = plt.subplots(1,2,figsize=(12,4))
ax[0].plot(history.history['loss'], label='train')
ax[0].plot(history.history['val_loss'], label='val')
ax[0].set_title('Loss')
ax[0].legend()
ax[1].plot(history.history['accuracy'], label='train')
ax[1].plot(history.history['val_accuracy'], label='val')
ax[1].set_title('Accuracy')
ax[1].legend()
plt.show()

In [ ]:
# Confusion matrix and classification report
y_pred_probs = model.predict(x_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = y_test.flatten()
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names, cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

print(classification_report(y_true, y_pred, target_names=class_names))

## Analysis of Findings

(Write your analysis here after running the notebook: discuss training vs validation performance, overfitting/underfitting, confusion matrix insights, and potential improvements.)

## References

- CIFAR-10 dataset: https://www.cs.toronto.edu/~kriz/cifar.html
- Keras Documentation: https://keras.io
- Goodfellow et al., Deep Learning (2016)